First, I'll show you the hard way, if you want to do all of this from scratch 

In [ ]:
import torch 
from torch_geometric.utils import erdos_renyi_graph
from torch_geometric.data import Data
from torch_cluster import random_walk

In [ ]:
NUM_NODES = 100
edge_index = erdos_renyi_graph(NUM_NODES, 0.1) # Generate random Erdos-Renyii graph w 100 nodes, and degree factor 0.1

# Top row is source nodes, bottom row is destination nodes
print(edge_index)

tensor([[ 0,  0,  0,  ..., 99, 99, 99],
        [23, 34, 35,  ..., 85, 90, 95]])


In [ ]:
# High-performance random walk generator 
from torch_cluster.rw import random_walk

# Need to add self loops, otherwise if rw generator gets to a terminal 
# node, when it tries to get the next edge probablility (1/num_edges) 
# it hits a divide by zero error and crashes
from torch_geometric.utils import add_remaining_self_loops
edge_index,_ = add_remaining_self_loops(edge_index)

WALK_LEN = 5
start_nodes = torch.arange(NUM_NODES)
src = edge_index[0]
dst = edge_index[1]
rws = random_walk(src, dst, start_nodes, WALK_LEN)
print(rws[:10])

tensor([[ 0, 78, 87, 88, 22, 59],
        [ 1, 21, 21, 11, 16, 33],
        [ 2, 67, 67,  4,  4, 30],
        [ 3, 56, 68, 56, 56, 24],
        [ 4, 27,  4,  4, 27, 82],
        [ 5,  5, 44,  5, 21, 38],
        [ 6, 92, 27,  4, 10,  4],
        [ 7, 71, 71, 65, 91,  5],
        [ 8, 75, 29, 88, 88, 63],
        [ 9, 66,  0, 78, 77, 78]])


In [ ]:
# Threw together simple skipgram model
from skipgram import SkipGram
from torch.optim import Adam

EPOCHS = 100
ENC_DIM = 16

model = SkipGram(NUM_NODES, ENC_DIM)
opt = Adam(model.parameters(), lr=0.01)
for e in range(EPOCHS):
    opt.zero_grad()
    rws = random_walk(*edge_index, torch.arange(NUM_NODES), WALK_LEN)
    batch, rw = rws[:,0], rws[:,1:]
    loss = model.forward(batch, rw)
    loss.backward()
    opt.step()

with torch.zero_grad():
    embs = model.embed(torch.arange(NUM_NODES))
    print(embs[:10])

AttributeError: module 'torch.nn' has no attribute 'module'